In [5]:
import pickle
from pathlib import Path
import pandas as pd

base_path = Path().resolve() 

relevant_hpos = pickle.load(open(base_path / 'files' / 'relevant_hpo_terms.pkl', 'rb'))

prompts_df = pd.read_excel(base_path / 'files' / 'prompts_omim_pmid_updated_file_final1.xlsx', index_col='File Name')
prompts_df = prompts_df.dropna()
prompts_df['OMIM'] = prompts_df['OMIM'].astype(int)
prompts_df

,Case Description,Correct Diagnosis,OMIM,PMID
File Name,,,,
PMID_10571775_KSN_II_1_en-prompt.txt,The proband was a female. Disease onset was no...,Distal renal tubular acidosis 4 with hemolytic...,611590,PMID_10571775_KSN_II_1_en-prompt.txt
PMID_10571775_YAT_II_1_en-prompt.txt,The proband was a male. Disease onset was not ...,Distal renal tubular acidosis 4 with hemolytic...,611590,PMID_10571775_YAT_II_1_en-prompt.txt
PMID_10580070_A_III_11_en-prompt.txt,The proband was a female. Disease onset occurr...,Cardiomyopathy dilated 1A,115200,PMID_10580070_A_III_11_en-prompt.txt
PMID_10580070_A_III_13_en-prompt.txt,The proband was a male. Disease onset occurred...,Cardiomyopathy dilated 1A,115200,PMID_10580070_A_III_13_en-prompt.txt
PMID_10580070_A_III_14_en-prompt.txt,The proband was a female. Disease onset occurr...,Cardiomyopathy dilated 1A,115200,PMID_10580070_A_III_14_en-prompt.txt
...,...,...,...,...
STX_Syrbe_3_en-prompt.txt,"The proband was a 5-year, 0-month old child. D...",Developmental and epileptic encephalopathy 4,612164,STX_Syrbe_3_en-prompt.txt
STX_Syrbe_4_en-prompt.txt,"The proband was a 6-year, 0-month old child. D...",Developmental and epileptic encephalopathy 4,612164,STX_Syrbe_4_en-prompt.txt
STX_Syrbe_5_en-prompt.txt,"The proband was a 4-year, 0-month old child. D...",Developmental and epileptic encephalopathy 4,612164,STX_Syrbe_5_en-prompt.txt


In [4]:
import os
import subprocess
import yaml
from tqdm.auto import tqdm

# Define the Exomiser JAR file path
EXOMISER_JAR = f"{Path().resolve().parent}/exomiser/exomiser-cli-14.0.0/exomiser-cli-14.0.0.jar"  
EXOMISER_DATA_DIR = f"{Path().resolve().parent}/exomiser/exomiser-cli-14.0.0/exomiser-data"  
OUTPUT_DIR = f"{base_path}/results/exomiser_results"  

# Ensure the output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

def run_exomiser(patient_name, hpo_terms):

    print(f"Processing {patient_name}...")

    if os.path.exists(f"{OUTPUT_DIR}/{os.path.splitext(patient_name)[0]}.csv"):
        return "Already processed"

    # Create a temporary YAML file for the patient
    yaml_data = {
        "analysis": {
            "genomeAssembly": "hg19", 
            "hpoIds": hpo_terms,
            "steps": [
                {"hiPhivePrioritiser": {}}
            ]
        },
        "outputOptions": {
            "numGenes": 0,
            "outputFormats": ["TSV_GENE"],  # We only need TSV for OMIM extraction
            "outputDirectory": OUTPUT_DIR,
            "outputFileName": f"{patient_name}_results"
        }
    }

    yaml_file = f"{OUTPUT_DIR}/{patient_name}.yml"
    with open(yaml_file, "w") as file:
        yaml.dump(yaml_data, file, default_flow_style=False)

    # Construct the Exomiser command with memory settings and data directory
    exomiser_command = [
        "java", "-Xms2g", "-Xmx4g",  # Memory allocation
        "-jar", EXOMISER_JAR,
        "--analysis", yaml_file,
        f"--exomiser.data-directory={EXOMISER_DATA_DIR}",
    ]

    print(f"Running Exomiser for {patient_name}...")
    # print(" ".join(exomiser_command))

    # subprocess.run(exomiser_command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Run Exomiser and print real-time logs
    process = subprocess.Popen(exomiser_command, cwd=f"{Path().resolve().parent}/exomiser/exomiser-cli-14.0.0", stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # for line in iter(process.stdout.readline, ''):
    #     print(line, end='')

    process.stdout.close()
    process.wait()

    if process.poll() is None:
        print(f"{patient_name}: Exomiser is still running...")
    else:
        print(f"{patient_name}: Exomiser has finished.")

    # Step 4: Extract OMIM IDs from the TSV result
    result_file = f"{OUTPUT_DIR}/{patient_name}_results.genes.tsv"

    if os.path.exists(result_file):
        df = pd.read_csv(result_file, sep="\t", low_memory=False)
        df["OMIM_IDs"] = df["HUMAN_PHENO_EVIDENCE"].str.extract(r'OMIM:(\d{6})')
        df = df.dropna(subset=["OMIM_IDs"])[["GENE_SYMBOL", "OMIM_IDs"]]
        df["Patient"] = patient_name  # Add patient name for reference
        df = df.head(20)
        df.to_csv(f"{OUTPUT_DIR}/{os.path.splitext(patient_name)[0]}.csv")

    # Step 5: Delete temporary files
    os.remove(yaml_file)
    if os.path.exists(result_file):
        os.remove(result_file)

    return "Done"

In [5]:
import concurrent.futures

# Run Exomiser in parallel
# Run Exomiser in parallel with progress tracking
with tqdm(total=len(prompts_df), desc="Running Exomiser", unit="patient") as progress_bar:
    with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
        futures = {executor.submit(run_exomiser, name, hpos): name for name, hpos in relevant_hpos.items()}

        for future in concurrent.futures.as_completed(futures):
            patient_name = futures[future]
            try:
                future.result()
                progress_bar.update(1)  # Increment progress bar when a patient is finished
                tqdm.write(f"{patient_name} completed.")
            except Exception as e:
                print(f"Error running Exomiser for {patient_name}: {e}")

Running Exomiser:   0%|          | 0/5145 [00:00<?, ?patient/s]

Processing PMID_34722527_individual_103_7_Hui_Wang_ComprehensiveM_en-prompt.txt...Processing PMID_32730804_Individual_3_en-prompt.txt...
Processing PMID_19776401_Patient_6_1_en-prompt.txt...
Processing PMID_20358596_Patient_A_en-prompt.txt...

Processing PMID_36586412_8_en-prompt.txt...
Processing PMID_36067010_Family_3_Patient_5_en-prompt.txt...
Processing PMID_19864672_Patient_1M_en-prompt.txt...
Processing PMID_26239674_proband_III_3_en-prompt.txt...
Processing PMID_11093277_K8345_III_3_IT_en-prompt.txt...
Processing PMID_34722527_individual_13_2_Moon_Woo_Seong_Molecularchara_en-prompt.txt...
Processing PMID_22541558_individual_R_1_en-prompt.txt...
Processing PMID_15266616_21_en-prompt.txt...
Processing PMID_30580808_Machol_Ind_3_Fam_33_en-prompt.txt...
Processing STX_G3_P3_en-prompt.txt...
Processing PMID_36446582_Ockeloen2015_P17_en-prompt.txt...
Processing PMID_37660254_Patient_1_en-prompt.txt...
🚀 Running Exomiser for PMID_37660254_Patient_1_en-prompt.txt...
Processing PMID_3373

Python(69803) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


✅ PMID_30400883_en-prompt.txt: Exomiser has finished.✅ PMID_26805782_F3_II_1_en-prompt.txt: Exomiser has finished.

Processing PMID_34722527_individual_individual_2_Li_Huang_Moleculargenet_en-prompt.txt...
✅ PMID_26805782_F3_II_1_en-prompt.txt completed.
🚀 Running Exomiser for PMID_34722527_individual_individual_2_Li_Huang_Moleculargenet_en-prompt.txt...
Processing PMID_36303223_individual_12_PMID_12112661_en-prompt.txt...
✅ PMID_30400883_en-prompt.txt completed.
🚀 Running Exomiser for PMID_36303223_individual_12_PMID_12112661_en-prompt.txt...


Python(69924) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(69925) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

✅ PMID_28148688_P1_en-prompt.txt: Exomiser has finished.
Processing PMID_31021519_individual_SATB2_112_en-prompt.txt...
✅ PMID_28148688_P1_en-prompt.txt completed.
🚀 Running Exomiser for PMID_31021519_individual_SATB2_112_en-prompt.txt...


Python(11939) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


✅ PMID_25439728_IV_6_en-prompt.txt: Exomiser has finished.
Processing PMID_33045405_Liu_et_al___2019__patient_1_en-prompt.txt...
🚀 Running Exomiser for PMID_33045405_Liu_et_al___2019__patient_1_en-prompt.txt...
✅ PMID_25439728_IV_6_en-prompt.txt completed.


Python(12460) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


✅ PMID_12789647_K12_I_2_en-prompt.txt: Exomiser has finished.
✅ PMID_1907800_TS_en-prompt.txt: Exomiser has finished.
✅ PMID_1907800_TS_en-prompt.txt completed.
✅ PMID_12789647_K12_I_2_en-prompt.txt completed.
✅ PMID_9817917_Family_4_individual_13070_en-prompt.txt: Exomiser has finished.
✅ PMID_9817917_Family_4_individual_13070_en-prompt.txt completed.
✅ PMID_28673863_28673863_P1_en-prompt.txt: Exomiser has finished.
✅ PMID_28673863_28673863_P1_en-prompt.txt completed.
✅ PMID_31021519_individual_SATB2_112_en-prompt.txt: Exomiser has finished.
✅ PMID_31021519_individual_SATB2_112_en-prompt.txt completed.
✅ PMID_33045405_Liu_et_al___2019__patient_1_en-prompt.txt: Exomiser has finished.
✅ PMID_33045405_Liu_et_al___2019__patient_1_en-prompt.txt completed.


In [3]:
# import pickle
# from pathlib import Path
# import pandas as pd

# base_path = Path().resolve() 

# file_name = "PMID_37154409_A_III_7_en-prompt"
# 'PMID_37154409_AIII-7-pheval_disease_result'
# file_name.replace('_en-prompt', 'pheval_disease_result')

# df = pd.read_csv(f"{base_path}/files/exomiser_14_response/{file_name}.tsv", sep='/t')
# df

In [1]:
from pathlib import Path
import pandas as pd 
import ast

base_path = Path().resolve() / 'files'

prompts_df = pd.read_excel(base_path / 'prompts_omim_pmid_updated_file_final1.xlsx', index_col='File Name')

prompts_df = prompts_df.dropna()
prompts_df['OMIM'] = prompts_df['OMIM'].astype(int)

# --- if the column is already a real list you can skip this helper ----
def ensure_list(x):
    """Turn stringified list / set into a real Python list."""
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        x = x.strip()
        if not x:
            return []
        try:                                     # try "[...]"  or "{...}"
            obj = ast.literal_eval(x)
            if isinstance(obj, (list, set, tuple)):
                return list(obj)
        except Exception:
            pass
        # fall-back: split on commas
        return [y.strip() for y in x.split(",") if y.strip()]
    return []

preds = pd.read_csv('all_predictions_with_hpo_prompts.csv')
preds["patient_hpo_names"] = preds["patient_hpo_names"].apply(ensure_list)
preds = preds[preds["patient_hpo_names"].apply(len) > 2].copy()

prompts_df = prompts_df.loc[preds['file_name'].unique()]

In [3]:
import pickle

with open(base_path / "depth_cache.pkl", "rb") as f:
    depth_cache = pickle.load(f)

In [6]:
from oaklib import get_adapter

adapter = get_adapter(f"simpleobo:{base_path}/mondo.obo")

In [7]:
import numpy as np
from utils.score_diagnosis import score_diagnosis
from tqdm.auto import tqdm
import os

OUTPUT_DIR = f"{base_path}/../results/exomiser_results"  

exact_match_ranks = []
exact_match_deepest_ancestor_match_ranks = []
for patient_name in tqdm(prompts_df.index):
    correct_diagnosis = prompts_df.loc[patient_name, 'OMIM']
    correct_diagnosis = f"OMIM:{correct_diagnosis}"

    # Fetch the gold standard ID
    results = adapter.sssom_mappings(correct_diagnosis)
    gold_standard_id = next(results, None)
    gold_standard_id = gold_standard_id.subject_id if gold_standard_id else None

    exact_match = False
    deepest_ancestor_match = False

    try: 
        df = pd.read_csv(f"{OUTPUT_DIR}/{os.path.splitext(patient_name)[0]}.csv")
    except Exception as e:
        print(f"Error reading {patient_name}: {e}")
        continue

    # Iterate through predictions
    for i, disease_id in enumerate(df['OMIM_IDs']):
        try:
            disease_id = f"OMIM:{disease_id}"

            score = score_diagnosis(adapter, disease_id, gold_standard_id, depth_cache=depth_cache)
            if score == "Exact match":
                exact_match_ranks.append(i+1)
                exact_match = True
                if not deepest_ancestor_match:
                    exact_match_deepest_ancestor_match_ranks.append(i+1)
                    deepest_ancestor_match = True
                break
            elif score == "Deepest ancestor match" and not deepest_ancestor_match:
                deepest_ancestor_rank = i+1
                exact_match_deepest_ancestor_match_ranks.append(deepest_ancestor_rank)
                deepest_ancestor_match = True
            else:
                pass
        except Exception as e:
            continue

    # Handle cases with no matches
    if not exact_match:
        exact_match_ranks.append(0)
    if not deepest_ancestor_match:
        exact_match_deepest_ancestor_match_ranks.append(0)

# Save the ranks to a file
np.savez(f"{base_path}/results/exomiser_results/ranks.npz", 
            exact_match_ranks=exact_match_ranks, 
            exact_match_deepest_ancestor_match_ranks=exact_match_deepest_ancestor_match_ranks)

  0%|          | 0/4419 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/timhulshof/Documents/analysis_chatimd/files/results/exomiser_results/ranks.npz'

In [8]:
# Save the ranks to a file
np.savez(f"{base_path}/../results/exomiser_results/ranks.npz", 
            exact_match_ranks=exact_match_ranks, 
            exact_match_deepest_ancestor_match_ranks=exact_match_deepest_ancestor_match_ranks)